In [ ]:
import sys, os
sys.path.append(os.pardir)
import urllib.request
import gzip
import numpy as np
from simple_convnet import SimpleConvNet
from common.optimizer import Adam
import matplotlib.pyplot as plt
from common.util import smooth_curve

# Fashion MNIST 데이터 다운로드
data_dir = os.path.join(os.getcwd(), 'data')
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

urls = ['http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz',
        'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz',
        'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz',
        'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz']

for url in urls:
    path = os.path.join(data_dir, url.split('/')[-1])
    urllib.request.urlretrieve(url, path)
    print(f'Downloaded {path}')

# Fashion MNIST 데이터 로드
def load_mnist(normalize=True, flatten=True, one_hot_label=False):
    data_dir = os.path.join(os.getcwd(), 'data')
    
    with gzip.open(os.path.join(data_dir, 'train-images-idx3-ubyte.gz'), 'rb') as f:
        train_images = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1, 28*28)
    with gzip.open(os.path.join(data_dir, 'train-labels-idx1-ubyte.gz'), 'rb') as f:
        train_labels = np.frombuffer(f.read(), np.uint8, offset=8)
        
    with gzip.open(os.path.join(data_dir, 't10k-images-idx3-ubyte.gz'), 'rb') as f:
        test_images = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1, 28*28)
    with gzip.open(os.path.join(data_dir, 't10k-labels-idx1-ubyte.gz'), 'rb') as f:
        test_labels = np.frombuffer(f.read(), np.uint8, offset=8)
        
    # Normalize
    if normalize:
        train_images = train_images / 255.0
        test_images = test_images / 255.0
        
    # Reshape
    if flatten:
        train_images = train_images.reshape(-1, 28, 28, 1)
        test_images = test_images.reshape(-1, 28, 28, 1)
    else:
        train_images = train_images.reshape(-1, 1, 28, 28)
        test_images = test_images.reshape(-1, 1, 28, 28)
        
    # One-hot labels
    if one_hot_label:
        train_labels = np.eye(10)[train_labels]
        test_labels = np.eye(10)[test_labels]
        
    return (train_images, train_labels), (test_images, test_labels)

# Fashion MNIST 데이터셋 로드
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True, flatten=False)

# SimpleConvNet 모델 생성
network = SimpleConvNet(input_dim=(1, 28, 28), 
                        conv_param={'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)

# Adam optimizer 생성 (학습률 0.01로 설정)
optimizer = Adam(lr=0.01, beta1=0.9, beta2=0.999, weight_decay=0.0)

# 하이퍼파라미터
iters_num = 10000  # 반복 횟수
train_size = x_train.shape[0]
batch_size = 100  # 미니배치 크기

# 학습 경과 기록
train_loss_list = []
train_acc_list = []
test_acc_list = []

# 1에폭 당 반복 수
iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    grads = network.gradient(x_batch, t_batch)
    
    # 가중치 업데이트
    optimizer.update(network.params, grads)
    
    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    # 1에폭마다 정확도 계산
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("epoch:", i // iter_per_epoch, "train acc:", train_acc, "test acc:", test_acc)

# 학습 진행 상황 그래프 그리기
iterations = range(0, iters_num)
plt.figure(figsize=(10, 6))
plt.plot(iterations, smooth_curve(train_loss_list), label='Training Loss')
plt.title('Training Loss')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

print("최종 훈련 데이터 정확도:", train_acc_list[-1])
print("최종 테스트 데이터 정확도:", test_acc_list[-1])

Downloaded C:\Users\shins\Desktop\과제\3\인공지능\data\train-images-idx3-ubyte.gz
Downloaded C:\Users\shins\Desktop\과제\3\인공지능\data\train-labels-idx1-ubyte.gz
Downloaded C:\Users\shins\Desktop\과제\3\인공지능\data\t10k-images-idx3-ubyte.gz
Downloaded C:\Users\shins\Desktop\과제\3\인공지능\data\t10k-labels-idx1-ubyte.gz
epoch: 0.0 train acc: 0.1 test acc: 0.1
epoch: 1.0 train acc: 0.8801833333333333 test acc: 0.87
epoch: 2.0 train acc: 0.8928166666666667 test acc: 0.876
epoch: 3.0 train acc: 0.9050666666666667 test acc: 0.88
epoch: 4.0 train acc: 0.9046 test acc: 0.8788
epoch: 5.0 train acc: 0.9114333333333333 test acc: 0.8866
